In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def plot_accuracy_loss(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Set hyperparameters
batch_size = 64
PATH = "../data/CUB_200_2011"
labels = pd.read_csv(os.path.join(PATH, "image_class_labels.txt"), sep=" ", header=None, names=["img_id", "label"])
images = pd.read_csv(os.path.join(PATH, "images.txt"), sep=" ", header=None, names=["img_id", "filepath"])
classes = pd.read_csv(os.path.join(PATH, "classes.txt"), sep=" ", header=None, names=["label", "category"])
num_classes = len(classes)
df = pd.merge(images, labels, on="img_id")
df = pd.merge(df, classes, on="label")
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Initialize transformations for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True, min_delta=0)
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/vgg16_EARLY.h5')
model = tf.keras.models.load_model('../models/vgg16_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True, min_delta=0)
num_epochs = 200
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/ResNet50_EARLY.h5')
model = tf.keras.models.load_model('model_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
leanring_rate = 0.0001

vgg16Model = VGG16(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(vgg16Model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg16Model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=leanring_rate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/vgg16.h5')
model = tf.keras.models.load_model('../models/vgg16.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
learning_rate = 0.0001
# Define the model
resnetModel = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(resnetModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=resnetModel.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
plot_accuracy_loss(history)

model.save('../models/ResNet50.h5')
model = tf.keras.models.load_model('model.h5')
model.evaluate(test_generator)

#Inception with 229*299 images

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

# Load the dataset and create generators
train_generator = train_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    dataframe=train_df,
    x_col="filepath",
    y_col="category",
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

test_generator = test_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=test_df,
    x_col="filepath",
    y_col="category",
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5], # Mimics ColorJitter's brightness adjustment
    channel_shift_range=0.2, # Partially mimics ColorJitter's hue adjustment
    fill_mode='nearest'
)

val_generator = val_datagen.flow_from_dataframe(
    directory=os.path.join(PATH, "images"),
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    dataframe=val_df,
    x_col="filepath",
    y_col="category",
)

In [ ]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel_EARLY = InceptionV3(weights='imagenet', include_top=False, min_delta=0)
x = GlobalAveragePooling2D()(inceptionModel_EARLY.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel_EARLY.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator)
plot_accuracy_loss(history)
model.save('../models/InceptionV3_EARLY.h5')

model = tf.keras.models.load_model('../models/InceptionV3_EARLY.h5')
model.evaluate(test_generator)

In [ ]:
num_epochs = 200
learning_rate = 0.0001
# Define the model
inceptionModel = InceptionV3(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inceptionModel.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inceptionModel.input, outputs=predictions)
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(train_generator, epochs=num_epochs,validation_data=val_generator)
plot_accuracy_loss(history)
model.save('../models/InceptionV3.h5')

model = tf.keras.models.load_model('../models/InceptionV3.h5')
model.evaluate(test_generator)